In [ ]:
import sys
import numpy as np, matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import display, Image

from pyhnc import *

In [ ]:
N = int(1e5)
Δr = 0.02
grid = Grid(N, Δr)
r, q = grid.r, grid.q

In [ ]:
T = 1.35
rho = 0.8

alpha = 0.5
niters = 1000
tol = 1e-12

hnc = HypernettedChainSolver(grid, alpha=alpha, niters=niters, tol=tol)
hnc.solve(potentials.LennardJones(), rho, T, monitor=True)
print(f'HNC pressure: {hnc.pressure/rho}\n')

py = PercusYevickSolver(grid)
py.solve(potentials.LennardJones(), rho, T, monitor=True)
print(f' PY pressure: {py.pressure/rho}')

plt.plot(r, py.g, label='PY')
plt.plot(r, hnc.g, '--', label='HNC')
plt.legend(loc='best')
plt.xlabel(r'$r$')
plt.ylabel(r'$g(r)$')
plt.xlim([0, 5])
plt.show()

## 2. Supercritical isotherms

Hansen & McDonald (2013) obtain the following equation of state for Lennard-Jones (presumably shifted and cut, but not 100%) along an isotherm above the critical temperature:

In [ ]:
Image('data/LJ_MSA_hansen2013.png')

In [ ]:
density = 0.7 * np.linspace(0, 1, 101)**0.5

p_py = np.ones_like(density)
p_hnc = np.ones_like(density)

v = potentials.LennardJones()
T = 1.35

monitor = False
# monitor = True

hnc = HypernettedChainSolver(grid)
py = PercusYevickSolver(grid)

for i, rho in enumerate(tqdm(density[1:])):
    try: p_hnc[i+1] = hnc.solve(v, rho, T, monitor=monitor).pressure / rho
    except: p_hnc[i+1] = np.nan
    try: p_py[i+1] = py.solve(v, rho, T, monitor=monitor).pressure / rho
    except: p_py[i+1] = np.nan

plt.figure(figsize=(3.375, 3))

plt.plot(density, p_py, label='PY(v)')
plt.plot(density, p_hnc, '--', label='HNC(v)')
plt.axhline(y=1, ls='dashed')

plt.legend(loc='best')
plt.xlabel(r'$\rho^*$')
plt.ylabel(r'$\beta p / \rho$')
plt.xlim([0, 0.8])
plt.ylim([0, 3])

plt.show()

# 3. Isotherms at coexistence

More challenging is below the critical temperature because the solutions become numerically unstable (due to physical inconsistencies) in the coexistence regime. Let's have a look at what we get.

First let's fix some parameters:

In [ ]:
v = potentials.LennardJones()
T = 0.8
monitor = False
# monitor = True

Below is the phase diagram from [wikipedia](https://en.wikipedia.org/wiki/Lennard-Jones_potential#/media/File:Vapor_liquid_equilibrium_properties_of_LJ_and_LJTS_potential.png).
* Blue: LJ shift/cut
* Black: full LJ.

Reading the blue lines (I used [this great extracting tool](https://automeris.io)), we expect to find coexistence densities along the isotherm $T^* = 0.8$ to be $\rho_-^* \simeq 0.019$ and $\rho_+^* \simeq 0.729$.

In [ ]:
rho_coexist = [0.019, 0.729]
Image('data/LJcut_phase_diagram_wikipedia.png')

Solving on the gas side of the boundary is straightforward:

In [ ]:
density_gas = 0.1 * np.linspace(0, 1, 251)

p_py_gas = np.ones_like(density_gas)
py = PercusYevickSolver(grid)

for i, rho in enumerate(tqdm(density_gas[1:])):
    try:
        p_py_gas[i+1] = py.solve(v, rho, T, monitor=monitor).pressure / rho
        if not py.converged: raise RuntimeError
    except:
        p_py_gas[i+1:] = np.nan
        print(rf'aborting - no solution above ρ={rho:.4g}')
        break

p_hnc_gas = np.ones_like(density_gas)
hnc = HypernettedChainSolver(grid)

for i, rho in enumerate(tqdm(density_gas[1:])):
    try:
        p_hnc_gas[i+1] = hnc.solve(v, rho, T, monitor=monitor).pressure / rho
        if not hnc.converged: raise RuntimeError
    except:
        p_hnc_gas[i+1:] = np.nan
        print(rf'aborting - no solution above ρ={rho:.4g}')
        break

print('done')

In [ ]:
plt.figure(figsize=(3.375, 3))

plt.plot(density_gas, p_py_gas * density_gas, '-', mfc='None', label='PY(v)')
plt.plot(density_gas, p_hnc_gas * density_gas, '--', mfc='None', label='HNC(v)')

plt.legend(loc='best')
plt.xlabel(r'$\rho^*$')
plt.ylabel(r'$\beta p^*$')
plt.xlim([0, 0.1])
plt.ylim([0, 0.05])

plt.axvline(x=rho_coexist[0], ls='dotted')

plt.show()

In [ ]:
rho0 = 0.8
density_liquid = np.linspace(rho0, 0.5, 1001)

p_hnc_liquid = np.empty_like(density_liquid)
p_py_liquid = np.empty_like(density_liquid)

hnc = HypernettedChainSolver(grid)
py = PercusYevickSolver(grid)

# We need a good initial guess to converge, so gradually move to coexistence
# regime in a path from low densities in the supercritical regime where
# solutions are more stable.
T0 = 1.35
nsteps = 5
for rho in np.linspace(0, rho0, nsteps)[1:]:
    py.solve(v, rho0, T0, monitor=monitor)
    hnc.solve(v, rho0, T0, monitor=monitor)
    assert py.converged
    assert hnc.converged

for T_tmp in np.linspace(T0, T, nsteps):
    py.solve(v, rho0, T_tmp, monitor=monitor)
    hnc.solve(v, rho0, T_tmp, monitor=monitor)
    assert py.converged
    assert hnc.converged

print('PY closure:')
for i, rho in enumerate(tqdm(density_liquid)):
    try:
        p_py_liquid[i] = py.solve(v, rho, T, monitor=monitor).pressure / rho
        if not py.converged: raise RuntimeError
    except:
        p_py_liquid[i:] = np.nan
        print(rf'aborting - no solution below ρ={rho:.4g}')
        break

print('HNC closure:')
for i, rho in enumerate(tqdm(density_liquid)):
    try:
        p_hnc_liquid[i] = hnc.solve(v, rho, T, monitor=monitor).pressure / rho
        if not hnc.converged: raise RuntimeError
    except:
        p_hnc_liquid[i:] = np.nan
        print(rf'aborting - no solution below ρ={rho:.4g}')
        break

print('done')

As can be seen, the solution fails as we approach the binodal densities. It would be hard to extract the binodal/spinodal points without improving the numerics approaching the phase boundary. Hopefully the proximity we can currently achieve will be good enough for subsequent sections.

In [ ]:
plt.figure(figsize=(3.375, 3))

pl, = plt.plot(density_liquid, p_py_liquid * density_liquid, '-', label='PY(v)')
plt.plot(density_gas, p_py_gas * density_gas, '-', c=pl.get_color())
pl, = plt.plot(density_liquid, p_hnc_liquid * density_liquid, '--', label='HNC(v)')
plt.plot(density_gas, p_hnc_gas * density_gas, '--', c=pl.get_color())

plt.text(0.5, 0.95, rf'$T^*={T:.1f}$',
         transform=plt.gca().transAxes, ha='center', va='top')

plt.legend(loc='best')
plt.xlabel(r'$\rho^*$')
plt.ylabel(r'$\beta p^*$')
plt.xlim([0, rho0])
plt.ylim([-0.1, 0.25])

plt.axvline(x=rho_coexist[0], ls='dotted')
plt.axvline(x=rho_coexist[1], ls='dotted')

plt.show()